# (1-old) Fit LINEAR TTVs

In [ ]:
with pm.Model() as lin_model:
    # stellar parameters (limb darkening using Kipping 2013)
    u = exo.distributions.QuadLimbDark('u', testval=np.array([U1,U2]))

    Rstar = pm.Bound(pm.Normal, lower=0)('Rstar', mu=RSTAR, sd=RSTAR_ERR)
    Mstar = pm.Bound(pm.Normal, lower=0)('Mstar', mu=MSTAR, sd=MSTAR_ERR)

    
    # planetary parameters (impact parameter using Espinoza 2018)
    logr = pm.Uniform('logr', lower=np.log(0.0003), upper=np.log(0.3), testval=np.log(radii), shape=NPL)
    rp   = pm.Deterministic('rp', T.exp(logr))
    
    b  = exo.distributions.ImpactParameter('b', ror=rp/Rstar, testval=impacts, shape=NPL)
    
    
    # polynomial TTV parameters    
    C0 = pm.Normal('C0', mu=0.0, sd=durations/2, shape=NPL)
    C1 = pm.Normal('C1', mu=0.0, sd=durations/2, shape=NPL)
    

    transit_times = []
    for npl in range(NPL):
        transit_times.append(pm.Deterministic('tts_{0}'.format(npl), \
                                              KEP_EPHEMERIS[npl] + \
                                              C0[npl]*Leg0[npl] + C1[npl]*Leg1[npl]))
    
    
    # set up stellar model and planetary orbit
    exoSLC = exo.StarryLightCurve(u)
    orbit  = exo.orbits.TTVOrbit(transit_times=transit_times, transit_inds=transit_inds, \
                                 b=b, r_star=Rstar, m_star=Mstar)
    
    # track period and epoch
    T0 = pm.Deterministic('T0', orbit.t0)
    P  = pm.Deterministic('P', orbit.period)
    
    
    # build the GP kernel using a different noise model for each season
    logSw4 = [None]*4
    logw0  = [None]*4
    logQ   = [None]*4
    
    kernel  = [None]*4
    
    for i in range(4):
        gpi = gp_priors[i]
        
        try:
            logSw4[i] = pm.Normal('logSw4_{0}'.format(i), mu=gpi['logSw4'][0], sd=gpi['logSw4'][1])
        except:
            logSw4[i] = gpi['logSw4'][0]
            
        try:
            logw0[i] = pm.Normal('logw0_{0}'.format(i), mu=gpi['logw0'][0], sd=gpi['logw0'][1])
        except:
            logw0[i] = gpi['logw0'][0]

        try:
            logQ[i] = pm.Normal('logQ_{0}'.format(i), mu=gpi['logQ'][0], sd=gpi['logQ'][1])
        except:
            logQ[i] = gpi['logQ'][0]

        
        kernel[i] = exo.gp.terms.SHOTerm(log_Sw4=logSw4[i], log_w0=logw0[i], log_Q=logQ[i])
        
        
    # nuissance parameters
    flux0 = pm.Normal('flux0', mu=1.0, sd=np.std(good_flux))

    if len(sc_flux_lin) > 1:
        logvar = pm.Normal('logvar', mu=np.log(np.var(sc_flux_lin)), sd=np.log(4))
    else:
        logvar = pm.Normal('logvar', mu=np.log(np.var(lc_flux_lin)*30), sd=np.log(4))
    
    
    # now evaluate the model for each quarter
    light_curves       = [None]*nq
    summed_light_curve = [None]*nq
    model_flux         = [None]*nq
    
    gp      = [None]*nq
    gp_pred = [None]*nq
    
    
    for j, q in enumerate(quarters):
        # set oversampling factor
        if all_dtype[q] == 'short':
            oversample = 1
        elif all_dtype[q] == 'long':
            oversample = 15
            
        # calculate light curves
        light_curves[j] = exoSLC.get_light_curve(orbit=orbit, r=rp, t=all_time[q], oversample=oversample)
        summed_light_curve[j] = pm.math.sum(light_curves[j], axis=-1) + flux0*T.ones(len(all_time[q]))
        model_flux[j] = pm.Deterministic('model_flux_{0}'.format(j), summed_light_curve[j])
        
        # here's the GP (w/ kernel by season)
        if all_dtype[q] == 'short':
            gp[j] = exo.gp.GP(kernel[q%4], all_time[q], T.exp(logvar)*T.ones(len(all_time[q])))
            
        elif all_dtype[q] == 'long':
            gp[j] = exo.gp.GP(kernel[q%4], all_time[q], T.exp(logvar)/30*T.ones(len(all_time[q])))
            
        else:
            raise ValueError("Cadence data type must be 'short' or 'long'")


        # add custom potential (log-prob fxn) with the GP likelihood
        pm.Potential('obs_{0}'.format(j), gp[j].log_likelihood(all_flux[q] - model_flux[j]))


        # track GP prediction
        gp_pred[j] = pm.Deterministic('gp_pred_{0}'.format(j), gp[j].predict())

In [ ]:
with lin_model:
    lin_map = exo.optimize(start=lin_model.test_point, vars=[flux0, logvar])
    lin_map = exo.optimize(start=lin_model.test_point, vars=[b])
    lin_map = exo.optimize(start=lin_map, vars=[u, Mstar])
    lin_map = exo.optimize(start=lin_map, vars=[C0, C1])
    
    for npl in range(NPL):
        try: lin_map = exo.optimize(start=lin_map, vars=[C2[npl], C3[npl]])
        except:
            try: lin_map = exo.optimize(start=lin_map, vars=[C2[npl]])
            except: pass

    lin_map = exo.optimize(start=lin_map)

In [ ]:
# grab the model flux and gp prediction
model_flux = []
gp_pred = []

for j, q in enumerate(quarters):
    model_flux.append(lin_map['model_flux_{0}'.format(j)])
    gp_pred.append(lin_map['gp_pred_{0}'.format(j)])


# plot the model lightcurve
plt.figure(figsize=(20,4))
for j, q in enumerate(quarters):
    plt.plot(all_time[q], all_flux[q], '.', color='lightgrey')
    plt.plot(all_time[q], model_flux[j] + gp_pred[j])
plt.show()

In [ ]:
# grab transit times and ephemeris
lin_transit_times = []
lin_ephemeris = []

for npl, p in enumerate(planets):
    lin_transit_times.append(lin_map['tts_{0}'.format(npl)])
    lin_ephemeris.append(lin_map['P'][npl]*transit_inds[npl] + lin_map['T0'][npl])
    
    p.kep_ephemeris = lin_map['P'][npl]*p.index + lin_map['T0'][npl]
    
    p.epoch  = lin_map['T0'][npl]
    p.period = lin_map['P'][npl]

KEP_EPHEMERIS = deepcopy(lin_transit_times)


# plot the OMC TTVs
fig, axes = plt.subplots(NPL, figsize=(12,8))

for npl, p in enumerate(planets):
    xtime = lin_transit_times[npl]
    yomc  = (lin_transit_times[npl] - lin_ephemeris[npl])*24*60
    
    axes[npl].plot(xtime, yomc, '.', c='C{0}'.format(npl))
    axes[npl].set_ylabel('O-C [min]', fontsize=20)
axes[NPL-1].set_xlabel('Time [BJKD]', fontsize=20)
plt.show()

In [ ]:
print('')
print('cumulative runtime = ', int(timer() - global_start_time), 's')
print('')

# (2 - old) Fit Independent TTVs using slide method

In [ ]:
rp = poly_map['rp']
b  = poly_map['b']

Rstar = poly_map['Rstar']
Mstar = poly_map['Mstar']
u = poly_map['u']

for npl, p in enumerate(planets):
    sma = get_sma(p.period, Mstar)
    dur = get_dur_tot(p.period, rp[npl], Rstar, b[npl], sma)
    
    p.duration = np.copy(dur)

In [ ]:
indep_transit_times = []
indep_ephemeris = []

slide_offset = 1.0
delta_chisq  = 2.0

for npl, p in enumerate(planets):
    poly_tts = np.copy(poly_transit_times[npl])
    poly_inds = np.copy(transit_inds[npl])
    
    slide_tts = np.zeros_like(poly_tts)
    slide_err = np.zeros_like(poly_tts)
        
    # create template transit
    exoSLC = exo.StarryLightCurve(u)
    orbit  = exo.orbits.KeplerianOrbit(t0=0, period=p.period, b=b[npl], r_star=Rstar, m_star=Mstar)
    
    gridstep = SCIT/3600/24/np.sqrt(5)
    
    template_time = np.arange(-p.duration*(slide_offset+1.5), p.duration*(slide_offset+1.5), gridstep)
    template_flux = 1.0 + exoSLC.get_light_curve(orbit=orbit, r=rp[npl], t=template_time).sum(axis=-1).eval()  
    
    
    for i, t0 in enumerate(poly_tts):
        time_stamp = lc.time[np.abs(lc.time - t0)/p.duration < (slide_offset+0.5)]
        flux_stamp = lc.flux[np.abs(lc.time - t0)/p.duration < (slide_offset+0.5)]
        error_stamp = lc.error[np.abs(lc.time - t0)/p.duration < (slide_offset+0.5)]
        
        if len(time_stamp) == 0:
            time_stamp = sc.time[np.abs(sc.time - t0)/p.duration < (slide_offset+0.5)]
            flux_stamp = sc.flux[np.abs(sc.time - t0)/p.duration < (slide_offset+0.5)]
            error_stamp = sc.error[np.abs(sc.time - t0)/p.duration < (slide_offset+0.5)]
            
                   
        tc_vector = t0 + np.arange(-p.duration*slide_offset, p.duration*slide_offset, gridstep)
        chisq_vector = np.zeros_like(tc_vector)

        # slide along transit time vector and calculate chisq
        for j, tc in enumerate(tc_vector):
            model_stamp = np.interp(time_stamp-tc, template_time, template_flux)
            
            chisq_vector[j] = np.sum((flux_stamp - model_stamp)**2/error_stamp**2)
            
            
        # grab points near minimum chisq
        min_chisq = chisq_vector.min()
        tcfit = tc_vector[chisq_vector < min_chisq+delta_chisq]
        x2fit = chisq_vector[chisq_vector < min_chisq+delta_chisq]

        # eliminate points far from the local minimum
        spacing = np.median(tcfit[1:]-tcfit[:-1])
        faraway = np.abs(tcfit-np.median(tcfit))/spacing > len(tcfit)/2

        tcfit = tcfit[~faraway]
        x2fit = x2fit[~faraway]
        
        # fit a parabola around the minimum (need at least 5 pts)
        if len(tcfit) < 5:
            slide_tts[i] = np.nan
            slide_err[i] = np.nan
        
        else:
            quad_coeffs = np.polyfit(tcfit, x2fit, 2)
            quadfit = np.polyval(quad_coeffs, tcfit)
            qtc_min = -quad_coeffs[1]/(2*quad_coeffs[0])
            qx2_min = np.polyval(quad_coeffs, qtc_min)
            qtc_err = np.sqrt(1/quad_coeffs[0])

            slide_tts[i] = np.mean([qtc_min,np.median(tcfit)])
            slide_err[i] = qtc_err            

            # check that the fit is well-conditioned (ie. a negative t**2 coefficient)
            if quad_coeffs[0] <= 0.0:
                slide_tts[i] = np.nan
                slide_err[i] = np.nan


            # check that the recovered transit time is within the expected range
            if (slide_tts[i] < tc_vector.min()) or (slide_tts[i] > tc_vector.max()):
                slide_tts[i] = np.nan
                slide_err[i] = np.nan
        
        #if ~np.isnan(slide_tts[i]):
        #    fig, axes = plt.subplots(1,2, figsize=(8,3))
        #    axes[0].plot(time_stamp, flux_stamp)
        #    axes[1].plot(tc_vector, chisq_vector, 'k.')
        #    axes[1].plot(tcfit, x2fit, 'r.')
        #    plt.show()
    
    # interpolate over bad points or overlapping transits  
    bad = np.isnan(slide_tts) + p.overlap[p.quality]
    out = np.abs(slide_tts-poly_tts)/astropy.stats.mad_std(slide_tts-poly_tts) > 8.0

    slide_tts[bad+out] = poly_tts[bad+out]    
    
    # save transit times and ephemeris
    indep_transit_times.append(slide_tts)

In [ ]:
fig, axes = plt.subplots(NPL, figsize=(12,8))

for npl, p in enumerate(planets):
    xtime = indep_transit_times[npl]
    yomc  = (indep_transit_times[npl] - poly_ephemeris[npl])*24*60
    
    axes[npl].plot(xtime, yomc, '.', c='C{0}'.format(npl))
    axes[npl].set_ylabel('O-C [min]', fontsize=20)
axes[NPL-1].set_xlabel('Time [BJKD]', fontsize=20)
plt.savefig(FIGURE_DIR + TARGET + '_ttvs_indep.pdf', bbox_inches='tight')
plt.show()

In [ ]:
print('')
print('cumulative runtime = ', int(timer() - global_start_time), 's')
print('')